In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from Preprocess import *
import time



In [ ]:
## Load data

processed_data              = pd.read_csv('feature_data.csv')  ## change path to point to data file
processed_data['timestamp'] = pd.to_datetime(processed_data['timestamp'])
label_index                 = [2,3]
feature_index               = np.arange(4,319)


In [ ]:
##Loop for saving data for multiple graph sizes and sequence lengths
data_folder     = Path("Graph_Folder") ## folder containing graphs processed by GEDD
sms_file_names  = [ "A_sms_local_10.pkl" ,"A_sms_local_15.pkl"] ## Add more filenames for corresponding graphs
graph_size      = np.array([10,15,20])
seq_arr         = [3,5]


which_graph     = 'sms'
train_date      =  processed_data.timestamp.min()  ## If want to impose time limit on training data
valid_date      =  processed_data.timestamp.max()
root            = Path(".")


for gr in range(0,graph_size.shape[0]):
    file_to_open       = data_folder/sms_file_names[gr]
    sms_file           = open(file_to_open,'rb')
    sms                = pickle.load(sms_file)
    num_users          = sms.A_msg[0].shape[0]
    norm_call, norm_sms = process_Graphs(sms,sms,scale=10)

    if (which_graph =='call'):
        graph = norm_call
    else:
        graph = norm_sms
    
    for seq in seq_arr:
        file_name   = "GraphSeq"+str(graph_size[gr])+"_"+str(seq)+".pkl"
        my_path     = root / "extracted_data" / file_name    ## folder for saving the data
        y_dict      = {}
        for lb in label_index:
            X_seq_a,X_cor_a,X_feat_a,y_a_lb, usr_arr_a = prepare_data_for_NN(processed_data,graph,seq,train_date,valid_date,feature_index,lb,num_users)
            y_dict["y_".format(lb) ]                   = y_a_lb
        
        data = {"X_seq_a":X_seq_a, "X_cor_a":X_cor_a, "X_feat_a":X_feat_a,"Y":y_dict,"usr_id":usr_arr_a }
        with my_path.open('wb') as fp:
            pickle.dump(data, fp)
    
    
    
    

